In [95]:
# Thêm vào đầu notebook sau phần import
KEYWORDS_MAP = {
    "ACB": ["ACB", "Ngân hàng ACB", "Ngân hàng TMCP Á Châu"],
    "BCM": ["BCM", "Becamex", "KCN Bình Dương", "khu công nghiệp Bình Dương", "VSIP", "Becamex IDC"],
    "BID": ["BIDV", "Ngân hàng Đầu tư và Phát triển Việt Nam"],
    "CTG": ["CTG", "VietinBank", "Ngân hàng Công Thương Việt Nam"],
    "DGC": ["DGC", "Hóa chất Đức Giang"],
    "FPT": ["FPT"],
    "GAS": ["PV GAS", "PV Gas", "Tổng Công ty Khí Việt Nam"],
    "GVR": ["GVR", "Tập đoàn Cao su", "Tập đoàn Công nghiệp Cao su Việt Nam"],
    "HDB": ["HDB", "HDBank", "Ngân hàng TMCP Phát triển Thành phố Hồ Chí Minh"],
    "HPG": ["HPG", "Hòa Phát"],
    "LPB": ["LPB", "LPBank", "LienVietPostBank", "Ngân hàng Bưu điện Liên Việt"],
    "MBB": ["MBB", "MBBank", "Ngân hàng Quân đội", "MB", "Ngân hàng TMCP Quân đội"],
    "MSN": ["MSN", "Masan", "WinCommerce"],
    "MWG": ["MWG", "Thế Giới Di Động", "Mobile World", "Bách Hóa Xanh", "BHX", "Điện Máy Xanh", "ĐMX", "TGDĐ"],
    "PLX": ["PLX", "Petrolimex", "Tập đoàn Xăng dầu Việt Nam"],
    "SAB": ["SAB", "Sabeco", "Tổng Công ty CP Bia - Rượu - Nước giải khát Sài Gòn"],
    "SHB": ["SHB", "Ngân hàng Thương mại Cổ phần Sài Gòn – Hà Nội", "Ngân hàng TMCP Sài Gòn Hà Nội"],
    "SSB": ["SSB", "Ngân hàng Thương mại Cổ phần Đông Nam Á", "Ngân hàng TMCP Đông Nam Á", "SeABank"],
    "SSI": ["SSI", "Chứng khoán SSI"],
    "STB": ["STB", "Sài Gòn Thương Tín", "Sacombank"],
    "TCB": ["TCB", "Techcombank", "Ngân hàng TMCP Kỹ Thương Việt Nam"],
    "TPB": ["TPB", "TPBank", "Ngân hàng Tiên Phong", "Ngân hàng TMCP Tiên Phong"],
    "VCB": ["VCB", "Vietcombank", "Ngân hàng TMCP Ngoại Thương Việt Nam", "Ngân hàng Ngoại thương"],
    "VHM": ["VHM", "Vinhomes"],
    "VIB": ["VIB", "Ngân hàng TMCP Quốc Tế Việt Nam", "Ngân hàng Quốc Tế"],
    "VIC": ["VIC", "Vingroup", "Công ty Cổ phần Tập đoàn Vingroup"],
    "VJC": ["VJC", "Vietjet Air", "Công ty Cổ phần Hàng không Vietjet", "máy bay Vietjet"],
    "VNM": ["VNM", "Vinamilk", "Công ty Cổ phần Sữa Việt Nam"],
    "VPB": ["VPB", "VPBank", "Ngân hàng TMCP Việt Nam Thịnh Vượng"],
    "VRE": ["VRE", "Vincom Retail", "Công ty Cổ phần Vincom Retail"]
}

def filter_articles_by_keywords(articles, keywords_map=KEYWORDS_MAP):
    """
    Lọc các bài báo có chứa từ khóa liên quan đến mã cổ phiếu VN30
    """
    filtered = []
    
    for art in articles:
        # Kết hợp title để tìm kiếm từ khóa
        text = art["title"].lower()
        matched_codes = []
        
        # Kiểm tra từng mã cổ phiếu
        for code, keywords in keywords_map.items():
            for keyword in keywords:
                if keyword.lower() in text:
                    matched_codes.append(code)
                    break  # Tìm thấy 1 keyword là đủ cho mã này
        
        # Chỉ giữ lại bài báo có ít nhất 1 mã cổ phiếu khớp
        if matched_codes:
            art_copy = art.copy()
            art_copy["codes"] = list(set(matched_codes))  # Loại bỏ trùng lặp
            filtered.append(art_copy)
    
    return filtered

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import json
from datetime import datetime

# Cấu hình Header giả lập trình duyệt thật (Rất quan trọng)
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Origin': 'https://vietstock.vn',
    'Referer': 'https://vietstock.vn/tai-chinh.htm',
}

def crawl_vietstock_category(url_category, max_pages=3, output_file=None):
    headers['Referer']=url_category
    session = requests.Session()
    articles_data = []
    
    # BƯỚC 1: Request trang gốc để lấy Cookies và ChannelID
    print(f"Đang truy cập trang gốc: {url_category}...")
    response = session.get(url_category, headers=headers)
    
    if response.status_code != 200:
        print("Không thể truy cập trang gốc.")
        return articles_data

    soup_main = BeautifulSoup(response.text, 'html.parser')
    
    # Lấy ChannelID tự động
    channel_id_tag = soup_main.find('input', {'id': 'hdfChannelID'}) or soup_main.find('input', {'id': 'channelID'})
    
    if channel_id_tag:
        channel_id = channel_id_tag.get('value')
        print(f"Đã tìm thấy Channel ID: {channel_id}")
    else:
        channel_id = '734' 
        print(f"Không tìm thấy Channel ID trong source, sử dụng mặc định: {channel_id}")

    # Lấy Token xác thực (nếu có trong form)
    token = ""
    token_tag = soup_main.find('input', {'name': '__RequestVerificationToken'})
    if token_tag:
        token = token_tag.get('value')

    # BƯỚC 2: Vòng lặp qua các trang (Pagination)
    api_url = "https://vietstock.vn/StartPage/ChannelContentPage"
    
    for page in range(1, max_pages + 1):
        print(f"\n--- Đang cào trang {page} ---")
        
        payload = {
            'channelID': channel_id,
            'page': page,
            '__RequestVerificationToken': token
        }
        
        try:
            post_response = session.post(api_url, data=payload, headers=headers)
            
            if post_response.status_code == 200:
                soup_api = BeautifulSoup(post_response.text, 'html.parser')
                articles = soup_api.find_all('h4', class_='channel-title') 
                
                if not articles:
                    articles = soup_api.select('h4 > a')

                for article in articles:
                    link_tag = article.find('a') if article.name != 'a' else article
                    
                    if link_tag:
                        title = link_tag.get('title') or link_tag.text.strip()
                        href = link_tag.get('href')
                        
                        if href and not href.startswith('http'):
                            full_link = f"https://vietstock.vn{href}"
                        else:
                            full_link = href
                        
                        # Tạo dictionary cho mỗi bài báo
                        article_data = {
                            'title': title,
                            'link': full_link,
                        }
                        articles_data.append(article_data)
                        
                        print(f"Tiêu đề: {title}")
                        print(f"Link: {full_link}")
                        
                        # Ghi từng dòng vào file JSONL
                        if output_file:
                            with open(output_file, 'a', encoding='utf-8') as f:
                                f.write(json.dumps(article_data, ensure_ascii=False) + '\n')
            else:
                print(f"Lỗi khi gọi API trang {page}: {post_response.status_code}")
                
        except Exception as e:
            print(f"Có lỗi xảy ra: {e}")
            
        time.sleep(2)
    
    return articles_data

# Chạy thử
vietstock_categories = [
    'https://vietstock.vn/chung-khoan.htm',
    'https://vietstock.vn/doanh-nghiep.htm',
    'https://vietstock.vn/bat-dong-san.htm',
    'https://vietstock.vn/tai-chinh.htm',
    'https://vietstock.vn/hang-hoa.htm',
    'https://vietstock.vn/kinh-te/vi-mo.htm',
    'https://vietstock.vn/kinh-te/kinh-te-dau-tu.htm',
    'https://vietstock.vn/the-gioi.htm',
    'https://vietstock.vn/dong-duong.htm',
    'https://vietstock.vn/tai-chinh-ca-nhan.htm',
    'https://vietstock.vn/nhan-dinh-phan-tich.htm',
    'https://vietstock.vn/san-giao-dich-tai-chinh.htm',
    'https://vietstock.vn/kinh-te.htm',
    'https://vietstock.vn/goc-nhin.htm',
]

# Tạo tên file với timestamp
output_filename = f'vietstock_articles.jsonl'

# Crawl tất cả các category
all_articles = []
for category_url in vietstock_categories:
    print(f"\n{'='*60}")
    print(f"Đang crawl category: {category_url}")
    print(f"{'='*60}")
    articles = crawl_vietstock_category(category_url, max_pages=2, output_file=output_filename)
    all_articles.extend(articles)
    time.sleep(3)

print(f"\n{'='*60}")
print(f"Hoàn thành! Đã lưu {len(all_articles)} bài báo vào file: {output_filename}")
print(f"{'='*60}")


Đang crawl category: https://vietstock.vn/chung-khoan.htm
Đang truy cập trang gốc: https://vietstock.vn/chung-khoan.htm...
Không tìm thấy Channel ID trong source, sử dụng mặc định: 734

--- Đang cào trang 1 ---
Tiêu đề: Ngân hàng Nhà nước: Tăng ưu tiên tín dụng nông nghiệp, giảm lãi suất và mở rộng hạn mức vay
Link: https://vietstock.vn/2025/11/ngan-hang-nha-nuoc-tang-uu-tien-tin-dung-nong-nghiep-giam-lai-suat-va-mo-rong-han-muc-vay-758-1374617.htm
Tiêu đề: MB bắt tay Visa, KOTRA ra mắt thẻ doanh nghiệp đa năng MB Visa Hi BIZ mới – “mở khoá” giao thương quốc tế cho doanh nghiệp
Link: https://vietstock.vn/2025/11/mb-bat-tay-visa-kotra-ra-mat-the-doanh-nghiep-da-nang-mb-visa-hi-biz-moi-8211-mo-khoa-giao-thuong-quoc-te-cho-doanh-nghiep-757-1374613.htm
Tiêu đề: Ngừng giao dịch ngân hàng bằng hộ chiếu từ 1/1/2026
Link: https://vietstock.vn/2025/11/ngung-giao-dich-ngan-hang-bang-ho-chieu-tu-112026-757-1374584.htm
Tiêu đề: Những hành vi bị nghiêm cấm trong hoạt động kiểm toán nội bộ Ngân hàn

In [106]:
# Đọc file JSONL và lọc theo keywords
print("\n" + "="*60)
print("Bắt đầu lọc bài báo theo keywords VN30...")
print("="*60)

all_articles_list = []

# Đọc từ file JSONL
with open(output_filename, 'r', encoding='utf-8') as f:
    for line in f:
        article = json.loads(line.strip())
        all_articles_list.append(article)

print(f"Tổng số bài báo đã crawl: {len(all_articles_list)}")

# Lọc theo keywords
filtered_articles = filter_articles_by_keywords(all_articles_list, keywords_map=KEYWORDS_MAP)
print(f"Số bài báo khớp với VN30 keywords: {len(filtered_articles)}")

# Lưu kết quả đã lọc vào file mới
filtered_filename = f'vietstock_articles_vn30_filtered.jsonl'

with open(filtered_filename, 'w', encoding='utf-8') as f:
    for article in filtered_articles:
        f.write(json.dumps(article, ensure_ascii=False) + '\n')

print(f"\n{'='*60}")
print(f"Đã lưu {len(filtered_articles)} bài báo VN30 vào file: {filtered_filename}")
print(f"{'='*60}")

# Thống kê theo mã cổ phiếu
code_stats = {}
for article in filtered_articles:
    for code in article['codes']:
        code_stats[code] = code_stats.get(code, 0) + 1

print("\nThống kê số bài báo theo mã cổ phiếu:")
for code, count in sorted(code_stats.items(), key=lambda x: x[1], reverse=True):
    print(f"{code}: {count} bài")


Bắt đầu lọc bài báo theo keywords VN30...
Tổng số bài báo đã crawl: 280
Số bài báo khớp với VN30 keywords: 56

Đã lưu 56 bài báo VN30 vào file: vietstock_articles_vn30_filtered.jsonl

Thống kê số bài báo theo mã cổ phiếu:
MBB: 28 bài
MWG: 14 bài
VPB: 14 bài
